# COMP 541 - Deep Learning Course Fall 2020
## Project
### Berk Güler - berkguler20@ku.edu.tr

In [3]:
import Pkg; Pkg.add("IterTools")
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using Statistics: mean
using Base.Iterators: flatten
using IterTools: ncycle, takenth
using Plots
using DelimitedFiles, Statistics, Random
dataset_directory_path = "../Tubitak ANN/DATA"
dataset_directory = filter(x -> isdir(joinpath(dataset_directory_path, x)), readdir(dataset_directory_path))

  Resolving package versions...
No Changes to `C:\Users\berkg\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\berkg\.julia\environments\v1.5\Manifest.toml`


7-element Array{String,1}:
 "doganay_experiment"
 "idil_experiment"
 "istenc_experiment"
 "omer_experiment"
 "ozan_experiment"
 "yahya_experiment"
 "zaid_experiment"

## Data Preprocessing

In [22]:
function split_and_convert(string_array)
    Rlength = length(string_array)
    Clength = length(split(string_array[1],","))
    Mdata = zeros(Float64,Rlength,Clength)
    for i in 1:Rlength
        row =  map(x->tryparse(Float64,x),split(string_array[i],","))
        Mdata[i,:] = row
    end
    return Mdata
end

split_and_convert (generic function with 1 method)

In [5]:
## Computing the Derivative of X to get Velocity
function take_derivative(x,t)
   v = zeros(size(t))
   for i in 2:length(t)
       velocity = (x[i] - x[i-1])/(t[i]-t[i-1])
       isnan(velocity) ? v[i] = 0 : v[i] = velocity
    end
    return v
end

take_derivative (generic function with 1 method)

## Read and Assign all of the data

In [30]:
test_data = Array{Float64}(undef,0,9)
train_data = Array{Float64}(undef,0,9)

for participant_index in 1:length(dataset_directory)
    #Participants: 1.doganay 2.idil 3.istenc 4.omer 5.ozan 6.yahya 7.zaid
    participant_directory_path = joinpath(dataset_directory_path,dataset_directory[participant_index])
    participant_directory = readdir(participant_directory_path)
    if participant_index != 5
        for raw_txt_files in participant_directory
            txt_file_directory_path = joinpath(participant_directory_path, raw_txt_files)
            txt_file_directory = readdir(txt_file_directory_path)

            raw_data_file = open(joinpath(txt_file_directory_path,txt_file_directory[4]))
            raw_data = split_and_convert(readlines(raw_data_file))
            close(raw_data_file)
            # Position Velocity F_fint F_int F_h m b a subtask 
            #     1        2       3     4    5  6 7 8    9       
            train_data = [train_data; raw_data]
        end
    else
        for raw_txt_files in participant_directory
            txt_file_directory_path = joinpath(participant_directory_path, raw_txt_files)
            txt_file_directory = readdir(txt_file_directory_path)

            raw_data_file = open(joinpath(txt_file_directory_path,txt_file_directory[4]))
            raw_data = split_and_convert(readlines(raw_data_file))
            close(raw_data_file)
            # Position Velocity F_fint F_int F_h m b a subtask 
            #     1        2       3     4    5  6 7 8    9       
            test_data = [test_data; raw_data]
        end
    end
end

In [36]:
time_history = 25;

25